## Leverage Document Grounding in Orchestration Service for RAG-based Content Generation

In this learning journey, you will learn how to leverage the Document Grounding module in the Orchestration Service to generate content using the Retrieval-Augmented Generation (RAG) approach.
The Document Grounding module helps in grounding the input questions to relevant documents.
The grounding process involves retrieving relevant documents from a knowledge base and using them to high-quality generate responses.
The knowledge base can be a collection of documents in a sharepoint folder, aws s3, an elastic search engine, or data repository which contains vectors.

In this learning journey, you will perform the following steps:
- Create the knowledge base with the relevant documents.
- Configure the Document Grounding module in the Orchestration Service.
- Generate content based on the knowledge base using the RAG approach.


## Prerequisites
Install the Generative AI Hub SDK using the following command:

In [ ]:
%pip install "sap-ai-sdk-gen[all]"

### Authenticating AI Core

In [ ]:
import json
import os 
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
# Inline credentials
with open('key.json') as f:
    credCF = json.load(f)

# Set environment variables
def set_environment_vars(credCF):
    env_vars = {
        'AICORE_AUTH_URL': credCF['url'] + '/oauth/token',
        'AICORE_CLIENT_ID': credCF['clientid'],
        'AICORE_CLIENT_SECRET': credCF['clientsecret'],
        'AICORE_BASE_URL': credCF["serviceurls"]["AI_API_URL"] + "/v2",
        'AICORE_RESOURCE_GROUP': "grounding" 
    }

    for key, value in env_vars.items():
        os.environ[key] = value
        print(value)

# Create AI Core client instance
def create_ai_core_client(credCF):
    set_environment_vars(credCF)  # Ensure environment variables are set
    return AICoreV2Client(
        base_url=os.environ['AICORE_BASE_URL'],
        auth_url=os.environ['AICORE_AUTH_URL'],
        client_id=os.environ['AICORE_CLIENT_ID'],
        client_secret=os.environ['AICORE_CLIENT_SECRET'],
        resource_group=os.environ['AICORE_RESOURCE_GROUP']
    )

ai_core_client = create_ai_core_client(credCF)

### Load the AI Core service key

In [10]:
key = json.load(open('key.json'))

In [11]:
AI_API_URL= key['serviceurls']['AI_API_URL']
clientid= key['clientid']
clientsecret= key['clientsecret']
url= key['url']
resource_group = "grounding"

### Fetch Token

This step is used to fetch an access token from the SAP AI Core OAuth server using client credentials grant type. The token will be used to authenticate API requests to AI Core (e.g., for creating secrets, executing workflows).

In [ ]:
import requests

headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'content-type': 'application/x-www-form-urlencoded',
}

data = f'grant_type=client_credentials&client_id={clientid}&client_secret={clientsecret}'

response = requests.post(f'{url}/oauth/token', headers=headers, data=data)

token = response.json()['access_token']

print(token)

### Resource Group

This step creates a new resource group in SAP AI Core and tags it with a label (document-grounding) to logically group related resources. The access token is used for authorized API access.

In [ ]:
headers = {
    'Authorization': f'Bearer {token}',
}

json_data = {
    'resourceGroupId': resource_group,
    'labels': [
        {
            'key': 'ext.ai.sap.com/document-grounding',
            'value': 'true',
        },
    ],
}

resource = requests.post(f'{AI_API_URL}/v2/admin/resourceGroups', headers=headers, json=json_data)

resource.json()

### configuration and Deployment

This step creates a configuration for an LLM orchestration scenario in SAP AI Core using the given executableId and scenarioId. The loop ensures the config is retried until it successfully returns a 201 Created status, handling transient errors.

In [14]:
headers = {
    'AI-Resource-Group': resource.json()['resourceGroupId'],
    'Authorization': f'Bearer {token}',
    "content-type": "application/json"
}

json_data = {
    'name': 'orchestration-config',
    'executableId': 'orchestration',
    'scenarioId': 'orchestration',
}

while True:
    configuration = requests.post(f'{AI_API_URL}/v2/lm/configurations', headers=headers, json=json_data)
    if(configuration.status_code == 201):
        break

This step deploys the LLM configuration. It then waits until the deployment is ready and retrieves the deploymentUrl(orchestration url), which is used to trigger orchestration requests.

In [ ]:
json_data = {
    'ttl': '24H',
    'configurationId': configuration.json()['id'],
}

response = requests.post(f'{AI_API_URL}/v2/lm/deployments', headers=headers, json=json_data)

response.json()

In [ ]:
while True:
    deployment = requests.get(f'{AI_API_URL}/v2/lm/deployments', headers=headers) 
    if(deployment.json()['resources'][0]['deploymentUrl'] != ''):
        break

deploymentUrl = deployment.json()['resources'][0]['deploymentUrl']

deploymentUrl

Here, you are explicitly defining the orchestration service deployment URL (orchestration_service_url) which points to your deployed LLM configuration. This URL is used to send inference requests (like prompt executions) to the SAP AI Core Orchestration.

In [ ]:
orchestration_service_url="https://api.ai.********************ondemand.com/v2/inference/deployments/dd525568f5e"

### Generic Secret

#### In this tutorial, we're demonstrating how to create a vector knowledge base by connecting either SharePoint or AWS S3 as the document source—multiple options are supported and optional based on your setup.

#### creating knowledge base using Sharepoint - option 1

This step specifically creates a secret in SAP AI Core that stores Base64-encoded credentials for SharePoint access, securely enabling document grounding workflows via Microsoft Graph.

In [ ]:
json_data = {
    'name': '<generic secret name>',
    'data': {
        'description': '<description of generic secret>',
        'clientId': '<client id>',
        'authentication': 'T0F1dGgyUGFzc3dvcmQ=',
        'tokenServiceURL': '<token service url>',
        'password': '<password>',
        'url': 'aHR0cHM6Ly9ncmFwaC5taWNyb3NvZnQuY29t',
        'tokenServiceURLType': 'RGVkaWNhdGVk',
        'user': '<user>',
        'clientSecret': '<client secret>',
        'scope': 'aHR0cHM6Ly9ncmFwaC5taWNyb3NvZnQuY29tLy5kZWZhdWx0',
    },
    'labels': [
        {
            'key': 'ext.ai.sap.com/document-grounding',
            'value': 'true',
        },
    ],
}

secret = requests.post(f'{AI_API_URL}/v2/admin/secrets', headers=headers, json=json_data)

secret.json()

#### creating knowledge base using AWS S3 - Option 2

Alternatively, instead of SharePoint, we can use AWS S3 as a document repository for grounding. In the example below, we securely store credentials as a secret named aws-s3-secret that will later be referenced in the pipeline creation.

This makes it clear that both SharePoint and AWS S3 are optional approaches and interchangeable based on the user’s infrastructure.

In [ ]:

# Prepare secret payload
secret_payload = {
    "name": "<generic secret name>",
    "data": {  
        "description": "<description of generic secret>",
        "url": "<url>",
        "authentication": "Tm9BdXRoZW50aWNhdGlvbg==",
        "access_key_id": "<access key id>",
        "secret_access_key": "<secret access key>",
        "bucket": "<bucket>",
        "region": "<region>",
        "host": "<host>",
        "username": "<username>"
    },
    "labels": [
        {
            "key": "ext.ai.sap.com/document-grounding",
            "value": "true"
        },
         {
            "key": "ext.ai.sap.com/documentRepositoryType",
            "value": "S3"
        }
    ]
}

# Create secret
response = requests.post(f"{AI_API_URL}/v2/admin/secrets", headers=headers, json=secret_payload)
print("Secret creation:", response.status_code, response.text)


### Pipeline Creation

#### Pipeline creation using sharepoint - option 1
In this step, we are creating a document grounding pipeline using SharePoint as the knowledge source. The pipeline connects to the document repository defined in the SharePoint site using the previously created secret 

In [ ]:
json_data = {
    'type': 'MSSharePoint',
    'configuration': {
        'destination': '<generic secret name>',
        'sharePoint': {
            'site': {
                'name': 'Dev_blr3_document',
                "includePaths": [
          "/sample_emails/output_texts"
        ]
            },
        },
    },
}

while True:
    pipeline = requests.post(f'{AI_API_URL}/v2/lm/document-grounding/pipelines', headers=headers, json=json_data)
    if(pipeline.status_code == 201):
        break

pipeline.json()['pipelineId']

#### Pipeline creation using AWS S3 - option 2
Once the secret (aws-s3-secret) is created, we can now configure the document grounding pipeline using AWS S3 as the data source. This example shows how to set up a pipeline by referencing the created secret. The pipeline will extract and prepare documents from the specified S3 bucket for grounding.

🔄 You can follow a similar flow for SharePoint or other supported sources — choosing between SharePoint and S3 is flexible based on your document storage setup.

In [ ]:
pipeline_payload = {
    "type": "S3",
    "configuration": {
        "destination": "<generic secret name>"
    }
}

# Create pipeline
response = requests.post(
    f"{AI_API_URL}/v2/lm/document-grounding/pipelines",
    headers=headers,
    json=pipeline_payload
)

if response.status_code == 201:
    pipeline_id = response.json().get("pipelineId")
    print("✅ Pipeline Created Successfully!")
    print("Pipeline ID:", pipeline_id)
else:
    print("❌ Pipeline creation failed:", response.status_code, response.text)


#### Set Up the Orchestration Service

Now that we have our document grounding pipeline ready, we can configure the LLM Orchestration Service to process incoming user queries in context.

We define a system message to describe the business scenario for the LLM — in this case, a Facility Solutions Company offering property maintenance and support services. The prompt template includes placeholders for the user’s query and the grounded document context (retrieved from S3 or SharePoint), making the responses personalized and context-aware.

💡 This setup ensures that the LLM generates accurate, domain-specific, and grounded responses using the extracted content from your enterprise documents.

In [ ]:
# Import libraries
from gen_ai_hub.proxy import get_proxy_client
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.document_grounding import (GroundingModule, DocumentGrounding, GroundingFilterSearch,
                                                                DataRepositoryType, DocumentGroundingFilter)
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService

In [24]:
# Set up the Orchestration Service
aicore_client = get_proxy_client().ai_core_client
orchestration_service = OrchestrationService(api_url=orchestration_service_url)
llm = LLM(
    name="gpt-4o",
    parameters={
        'temperature': 0.0,
    }
)
template = Template(
            messages=[
                SystemMessage("""Facility Solutions Company provides services to luxury residential complexes, apartments,
                individual homes, and commercial properties such as office buildings, retail spaces, industrial facilities, and educational institutions.
                Customers are encouraged to reach out with maintenance requests, service deficiencies, follow-ups, or any issues they need by email.
                """),
                UserMessage("""You are a helpful assistant for any queries for answering questions.
                Answer the request by providing relevant answers that fit to the request.
                Request: {{ ?user_query }}
                Context:{{ ?grounding_response }}
                """),
            ]
        )

In [ ]:
# Set up Document Grounding
filters = [DocumentGroundingFilter(id="vector",
                                   data_repositories=["52********************fc2c"],
                                   search_config=GroundingFilterSearch(max_chunk_count=2),
                                   data_repository_type=DataRepositoryType.VECTOR.value
                                   )
]

grounding_config = GroundingModule(
            type="document_grounding_service",
            config=DocumentGrounding(input_params=["user_query"], output_param="grounding_response", filters=filters)
        )

config = OrchestrationConfig(
    template=template,
    llm=llm,
    grounding=grounding_config
)

 #### Step 3: Generate context-relevant answer for a user query
   - We now invoke the orchestration service by providing a user query. The query is grounded against the document index, and the LLM uses the grounding result to generate an informed response.

In [26]:
response = orchestration_service.run(config=config,
                            template_values=[
                                TemplateValue("user_query", "Is there any complaint?"),
                            ])
print(response.orchestration_result.choices[0].message.content)

Based on the context provided, there are two issues that could be considered complaints:

1. **HVAC Noise Issue**: Robert Kim from Lakeview Corporate Offices has expressed a concern about a minor noise issue following the repair of their HVAC system. He is requesting a technician to look into this matter.

2. **Pothole on Victoria Rd**: A concerned citizen has reported a pothole on 27-3 Victoria Rd, highlighting the potential inconvenience and danger it poses to pedestrians and drivers. The citizen is urging the public administration to address this issue urgently.

Both of these can be considered complaints or requests for further action to resolve the issues mentioned.
